<a href="https://colab.research.google.com/github/akira234mirai/AlineRocha/blob/akira234mirai-patch-1/DeepLearning_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rede Profunda

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #Transformar classe em números
from sklearn.feature_extraction.text import CountVectorizer #Cria o Vetor com o texto
from sklearn.metrics import confusion_matrix
from keras.models import Sequential #Conjunto de camadas sequenciais
from keras.layers import Dense, Dropout
from google.colab import files

In [ ]:
files.upload()

In [ ]:
spam = pd.read_csv('spam.csv')
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
#transformar as categorias (classes) em numérico
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam['Category'])
print(y)

[0 0 1 ... 0 0 0]


In [ ]:
mensagens = spam['Message'].values #Transforma objeto do pandas para numpy

X_train, X_test, y_train, y_test = train_test_split(mensagens, y, test_size=0.3)
print(X_train)

['Fine i miss you very much.'
 'Shuhui say change 2 suntec steamboat? U noe where? Where r u now?'
 "K, I'll work something out" ...
 'Its a site to simulate the test. It just gives you very tough questions to test your readiness.'
 "What u talking bout early morning? It's almost noon where your at!"
 'Sending you greetings of joy and happiness. Do have a gr8 evening']


In [ ]:
vetorizador = CountVectorizer() #Cria vetor de 0 e 1 para cada palavra (OneHot Encoding)
vetorizador.fit(X_train) #Fit: cria o modelo
X_train = vetorizador.transform(X_train)
X_test = vetorizador.transform(X_test)
print(X_train.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)

In [ ]:
X_train.toarray()[0]

In [ ]:
print(X_train.shape[1])

7121


In [ ]:
modelo = Sequential() #Adiciona as camadas
#Camada de entrada
#units: neuronios, activation: função ativação. input_dim: qtd de colunas
modelo.add(Dense(units=10, activation='relu', input_dim=X_train.shape[1]))
#Técnica para minimizar overfiting
#Remove aleatóriamente 1% de sinapses (conexões)
modelo.add(Dropout(0.1))
#2ª camada oculta densa
#nª de neurônios ser referente ao número de classes
modelo.add(Dense(units=8, activation='relu'))
modelo.add(Dropout(0.1))
#Última camada
#Como vai retornae somente 1 valor, units=1; Muda função de ativação
modelo.add(Dense(units=1, activation='sigmoid'))


In [ ]:
#Compilar a rede
#Função loss: função de perda
#Optimizer: Ajuste dos pesos. 'adam': estocástico
#Metrics: Avaliar a performance
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                71220     
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 71317 (278.58 KB)
Trainable params: 71317 (278.58 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#dados de treino
#epochs: serão passados todos os dados de treino pela rede
#batch_size: A cada qtds registros atualiza-se a rede?
#verbose: retorno do processo iterativo de treinamento
#validation_data: Treina e usa dados de validação para melhorar o modelo

modelo.fit(X_train, y_train, epochs=20, batch_size=10, verbose=True, validation_data=(X_test, y_test))

Epoch 1/20
390/390 [==============================] - 4s 6ms/step - loss: 0.0937 - accuracy: 0.9305 - val_loss: 0.0215 - val_accuracy: 0.9761
Epoch 2/20
390/390 [==============================] - 1s 3ms/step - loss: 0.0160 - accuracy: 0.9872 - val_loss: 0.0141 - val_accuracy: 0.9803
Epoch 3/20
390/390 [==============================] - 1s 3ms/step - loss: 0.0088 - accuracy: 0.9918 - val_loss: 0.0127 - val_accuracy: 0.9844
Epoch 4/20
390/390 [==============================] - 1s 3ms/step - loss: 0.0055 - accuracy: 0.9946 - val_loss: 0.0129 - val_accuracy: 0.9839
Epoch 5/20
390/390 [==============================] - 1s 4ms/step - loss: 0.0036 - accuracy: 0.9974 - val_loss: 0.0127 - val_accuracy: 0.9850
Epoch 6/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0038 - accuracy: 0.9967 - val_loss: 0.0146 - val_accuracy: 0.9809
Epoch 7/20
390/390 [==============================] - 1s 3ms/step - loss: 0.0024 - accuracy: 0.9979 - val_loss: 0.0151 - val_accuracy: 0.9803
Epoch 

In [ ]:
loss, accuracy = modelo.evaluate(X_test, y_test)
print('Loss:', loss)
print('Acurácia:', accuracy)

53/53 [==============================] - 0s 2ms/step - loss: 0.0132 - accuracy: 0.9844
Loss: 0.013239754363894463
Acurácia: 0.9844497442245483


In [ ]:
nova_previsao = modelo.predict(X_test)
print(nova_previsao)

In [ ]:
prev = (nova_previsao > 0.5)
print(prev)

In [ ]:
cm = confusion_matrix(y_test, prev)
print(cm)

[[1438    0]
 [  26  208]]


Rede Neural 2 - Deep Learning

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from google.colab import files
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
files.upload()

In [ ]:
spam = pd.read_csv('spam.csv')
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam['Category'])
print(y)

In [ ]:
mensagens = spam['Message'].values #Transforma objeto do pandas para numpy

X_train, X_test, y_train, y_test = train_test_split(mensagens, y, test_size=0.3)

In [ ]:
# 1 - tokenizar o texto
token = Tokenizer(num_words=1000) #Nº maximo de palavras a serem mantidas
print(token)

In [ ]:
#Fit: transforma
token.fit_on_texts(X_train)
#Criação da matriz densa dos tokens
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [ ]:
#Matrix densa dos tokens
#Transformou palavras em números
print(X_train)

[[57, 77, 64, 82, 205, 10, 27], [769, 71, 4, 770, 357, 247, 3, 16, 118, 488, 2, 136, 829, 2, 119, 15, 119, 430, 476, 726, 567, 60], [45], [170, 239, 830, 1, 100, 49, 373, 2, 167, 177, 334, 173, 3, 431, 598], [28, 3, 70, 4, 108, 209, 477, 105, 164, 34, 47, 358, 2, 101, 93, 683, 47, 68, 47, 684, 95, 30, 15], [85, 79, 15, 97], [147, 83, 6, 149], [1, 26, 67, 156, 2, 13, 568, 28, 3, 295, 43, 42, 478, 227, 124], [71, 8, 830], [685, 112, 197, 17, 156, 541, 569, 65, 359, 686], [32, 85, 228, 5, 17, 305, 20, 98, 178, 8, 570, 3, 727, 4, 111, 2, 10, 3, 16, 118, 4, 296, 27, 50, 478, 257, 27, 111, 407, 28, 16, 4, 111, 116], [345, 432, 374, 274, 728, 283, 306, 729, 41, 47, 135, 13, 105, 929, 15, 47, 18, 2, 453, 21], [514, 33, 730, 515, 6, 831, 91, 14, 542, 307, 730, 9, 125, 454, 82, 433, 141, 17, 930, 181, 82, 388, 16, 229, 62, 109], [1, 25, 335, 9, 72], [316, 288, 21, 1, 58, 4, 206, 10, 186, 288, 35, 1, 135, 4, 264, 206, 10, 19, 53], [6, 137, 193, 137, 289, 17, 33, 317], [51, 102, 15, 71, 72, 9, 142

In [ ]:
#Garantir que todos os tokens tenham a mesma quantidade de elementos numéricos
#Preenche com zeros depois do final da numeração
#Tamanho maximo: 500 caracters
X_train = pad_sequences(X_train, padding = 'post', maxlen=500)
X_test = pad_sequences(X_test, padding = 'post', maxlen=500)

In [ ]:
#Dados em forma tabular
print(X_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(len(token.word_index))

7389


In [ ]:
modelo = Sequential()
#Camada Embedding:
#input_dim: tamanho vocabulário
#Output_dim: tamanho do vetor denso
#input_length: comprimento da sequencia

modelo.add(Embedding(input_dim=len(token.word_index), output_dim=50, input_length=500))
modelo.add(Flatten()) #Conecta camada Embedding com camada densa

modelo.add(Dense(units=10, activation='relu'))
modelo.add(Dropout(0.1))
modelo.add(Dense(units=1, activation='sigmoid'))

In [ ]:
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
modelo.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           369450    
                                                                 
 flatten (Flatten)           (None, 25000)             0         
                                                                 
 dense_3 (Dense)             (None, 10)                250010    
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
                                                                 
Total params: 619471 (2.36 MB)
Trainable params: 619471 (2.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
modelo.fit(X_train, y_train, epochs=20, batch_size=10, verbose=True, validation_data=(X_test, y_test))

In [ ]:
loss, accuracy = modelo.evaluate(X_test, y_test)
print('Loss:', loss)
print('Acurácia:', accuracy)

53/53 [==============================] - 0s 3ms/step - loss: 0.0161 - accuracy: 0.9827
Loss: 0.016135362908244133
Acurácia: 0.9826555252075195


In [ ]:
nova_previsao = modelo.predict(X_test)
print(nova_previsao)

In [ ]:
prev = (nova_previsao > 0.5)
print(prev)

In [ ]:
cm = confusion_matrix(y_test, prev)
print(cm)

[[1447   12]
 [  17  196]]
